In [1]:
!pip install datasets
!pip install transformers[torch]
!pip install accelerate -U

In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
import torch
from torch.utils.data import Dataset, DataLoader
import torchaudio
import os
import datasets
from datasets import load_dataset
from transformers import Wav2Vec2ForPreTraining, Wav2Vec2Processor, TrainingArguments, Trainer

In [3]:
model = Wav2Vec2ForPreTraining.from_pretrained("facebook/wav2vec2-base")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav

In [18]:
dataset = load_dataset("koyealagbe/nnces-unlabeled")
split_dataset = dataset['train'].train_test_split(test_size=0.9)

# Use only one half
dataset_half = split_dataset['train']  # You can choose 'train' or 'test' as per your need

# Verify the split
print(dataset_half)

Resolving data files:   0%|          | 0/9510 [00:00<?, ?it/s]

Dataset({
    features: ['audio'],
    num_rows: 951
})


In [34]:
def preprocess_function(batch):
    audio = torch.tensor(batch["audio"]["array"]).to(torch.float)
    if len(audio) == 0:
        audio = torch.zeros(44100 * 5).to(torch.float)  # Placeholder for empty audio
    resampler = torchaudio.transforms.Resample(orig_freq=44100, new_freq=16000)
    audio_16k = resampler(audio)
    inputs = processor(audio_16k, sampling_rate=16000, return_tensors="pt", padding=True)
    return inputs

dataset = dataset_half.map(preprocess_function).remove_columns(["audio"])
print(dataset)

Map:   0%|          | 0/951 [00:00<?, ? examples/s]

Dataset({
    features: ['input_values'],
    num_rows: 951
})


In [37]:
def collate_fn(batch):
    try:
        print(batch[0])
        input_values = [torch.tensor(item['audio']['array']) for item in batch]
        return {
            'input_values': torch.stack(input_values)
        }
    except KeyError as e:
        print(f"KeyError: {e}")
        print(f"Batch keys: {[item.keys() for item in batch]}")
        raise

train_dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

In [38]:
training_args = TrainingArguments(
    output_dir="./wav2vec2-pretraining",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    remove_unused_columns=False,
    dataloader_num_workers=4,
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    train_dataset=dataset_half,
    tokenizer=processor.feature_extractor
)

# Train the model
trainer.train()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/cc7e8d7b69ee344fa6e38c3c67bdc52820c80dfa22ced01dead22253ef0e94fd', 'array': array([-0.00164795, -0.00167847, -0.00177002, ..., -0.0496521 ,
       -0.05090332, -0.05175781]), 'sampling_rate': 44100}}
{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/aadace04cc106de5fe861717d8f6bb1a6aa6839e155f00a570f50647333a627d', 'array': array([0.00497437, 0.00622559, 0.00753784, ..., 0.00286865, 0.00305176,
       0.00320435]), 'sampling_rate': 44100}}
{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/c79b0bf6483de5288c074d4aa1654e7b17b1be27905375d78d69147654f191d6', 'array': array([ 0.00192261,  0.00186157,  0.0020752 , ..., -0.00082397,
       -0.00067139, -0.00048828]), 'sampling_rate': 44100}}{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/c20d756c912b09fa6c77cd6859b65ecf35da2d32ea524bc1204c65f7479d647b', 'array': array([ 0.00057983,  0.00109863,  0.00057983, ..., -0.00140381,
       -0.0

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
  File "<ipython-input-37-8a6097350f29>", line 6, in collate_fn
    'input_values': torch.stack(input_values)
RuntimeError: stack expects each tensor to be equal size, but got [311296] at entry 0 and [176128] at entry 1


{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/3916bd9e4bd5ae03274a0d1e374952fa5abd2986503d8d7c7c1fe418d7bdbb29', 'array': array([-3.05175781e-04, -3.05175781e-05,  1.52587891e-04, ...,
        1.49536133e-03,  1.80053711e-03,  1.49536133e-03]), 'sampling_rate': 44100}}
{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/05d56a2b1be3ce1b62a9810530c8024419e99683a31a22a9f88e6272e1a4e445', 'array': array([ 0.00030518,  0.00048828,  0.00073242, ..., -0.00900269,
       -0.01077271, -0.01208496]), 'sampling_rate': 44100}}
{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/93dc1b43113d23d0ff77a87b428b84414f1b3c650654abd685587fee8a06b9e5', 'array': array([-0.00463867, -0.00515747, -0.00570679, ...,  0.00387573,
        0.0043335 ,  0.00466919]), 'sampling_rate': 44100}}
{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/1dec1f863731d50bf19512e0cd8afe1636a00ee2bcd972fbd2681b528fc93edb', 'array': array([ 0.00100708,  0.00088501,  0.0007019 

### 1. Make Labeled Dataset

1. Load Audio Transcription Data

In [7]:
transcriptions = pd.read_csv('transcriptions.csv')
transcriptions.head()

,Filepath,Filename,Transcript
0,Read_Speech_Data/Read_Speech_Data/8Y/Boys_8/M1...,M1_01_01.wav,The oldest member of the family is my grandmot...
1,Read_Speech_Data/Read_Speech_Data/8Y/Boys_8/M1...,M1_01_02.wav,"20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9,..."
2,Read_Speech_Data/Read_Speech_Data/8Y/Boys_8/M1...,M1_01_03.wav,Mom and Dad arrived at the bus station early b...
3,Read_Speech_Data/Read_Speech_Data/8Y/Boys_8/M1...,M1_01_04.wav,Dog eats bones. Mike likes bikes. Elsa wants a...
4,Read_Speech_Data/Read_Speech_Data/8Y/Boys_8/M1...,M1_01_05.wav,I don't have time to go with you. I am playing...


2. Make Custom Dataset for Labeled Data

In [8]:
class LabeledDataset(Dataset):
  def __init__(self, transcription_df, transform=None):
    self.df = transcription_df
    self.transform = transform

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    row = self.df.iloc[idx]
    path = row['Filepath']
    name = row['Filename']
    file = 'archive/' + path + '/' + name
    transcript = row['Transcript']
    wf, sr = torchaudio.load(file)

    return wf, transcript

labeled_data = LabeledDataset(transcriptions) # SR: 44100 Hz
print(len(labeled_data))
print(labeled_data[5])

500
(tensor([[ 0.0039,  0.0033,  0.0033,  ..., -0.0005, -0.0002, -0.0002],
        [ 0.0039,  0.0033,  0.0033,  ..., -0.0005, -0.0002, -0.0002]]), 'The nature is very attractive and full of my favorite green color.')


### 2. Load Unlabeled Audio

In [9]:
### 2. Load unlabeled audio into Pytorch Tensor
from transformers import Wav2Vec2Processor, Wav2Vec2ForPreTraining, Trainer, TrainingArguments

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")

class UnlabeledDataset(Dataset):
    def __init__(self, root_dir, target_sample_rate=16000, processor=None):
        self.root_dir = root_dir
        self.target_sample_rate = target_sample_rate
        self.file_paths = []
        self.load_file_paths()
        self.processor = processor
        self.resampler = torchaudio.transforms.Resample(orig_freq=44100, new_freq=self.target_sample_rate)

    def load_file_paths(self):
        for root, _, files in os.walk(self.root_dir):
            for file in files:
                if file.endswith('.wav'):
                    file_path = os.path.join(root, file)
                    self.file_paths.append(file_path)


    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        wf, sr = torchaudio.load(file_path)
        if sr != self.target_sample_rate:
            wf = self.resampler(wf)
        # pad audio
        wf = wf.squeeze()
        wf_length = wf.size(0)
        if wf_length < 960000:
            # Pad with zeros if shorter than a minute
            padding = 960000 - wf_length
            wf = torch.nn.functional.pad(wf, (0, padding))
        else:
            print('hi')
            # Truncate if longer than a minute
            wf = wf[:960000]

        return wf

unlabeled_data = UnlabeledDataset('archive', processor=processor)
print(len(unlabeled_data))
print(unlabeled_data[10])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

10051
tensor([[-0.0018, -0.0041, -0.0049,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0018, -0.0041, -0.0049,  ...,  0.0000,  0.0000,  0.0000]])


### 3. Train Wav2Vec 2.0 Model from Hugging Face

In [10]:
model = Wav2Vec2ForPreTraining.from_pretrained("facebook/wav2vec2-base")

def collate_fn(batch):
    inputs = processor(batch, sampling_rate=16000, return_tensors="pt", padding=True)
    return {'input_values': inputs.input_values}

dataloader = DataLoader(unlabeled_data, batch_size=8)


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForPreTraining: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForPreTraining from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForPreTraining from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForPreTraining were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should prob

In [11]:
'''
def prepare_data(batch):
  audio = batch["speech"]
  sampling_rate = batch["sampling_rate"]
  inputs = processor(audio, sampling_rate=sampling_rate, return_tensors="pt", padding=True)
  return {'input_values': inputs.input_values.squeeze(0), 'sampling_rate': sampling_rate}

def collate_fn(batch):
  batch = [prepare_data(item) for item in batch]
  input_values = torch.nn.utils.rnn.pad_sequence([item['input_values'] for item in batch], batch_first=True)
  sampling_rates = torch.tensor([item['sampling_rate'] for item in batch])
  return {'input_values': input_values, 'sampling_rate': sampling_rates}

dataloader = DataLoader(unlabeled_data, batch_size=8, collate_fn=collate_fn)
'''

'\ndef prepare_data(batch):\n  audio = batch["speech"]\n  sampling_rate = batch["sampling_rate"]\n  inputs = processor(audio, sampling_rate=sampling_rate, return_tensors="pt", padding=True)\n  return {\'input_values\': inputs.input_values.squeeze(0), \'sampling_rate\': sampling_rate}\n\ndef collate_fn(batch):\n  batch = [prepare_data(item) for item in batch]\n  input_values = torch.nn.utils.rnn.pad_sequence([item[\'input_values\'] for item in batch], batch_first=True)\n  sampling_rates = torch.tensor([item[\'sampling_rate\'] for item in batch])\n  return {\'input_values\': input_values, \'sampling_rate\': sampling_rates}\n\ndataloader = DataLoader(unlabeled_data, batch_size=8, collate_fn=collate_fn)\n'

3. Train the model

In [11]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./wav2vec2",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    save_steps=10_000,
    logging_steps=500,
    learning_rate=1e-4,
    weight_decay=0.01,
    fp16=True
)

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=training_args.learning_rate)

# Training loop
model.train()
for epoch in range(training_args.num_train_epochs):
    i = 1
    for batch in dataloader:
        print(i)
        inputs = batch['input_values'].to('cuda')
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        i += 1
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

model.save_pretrained(training_args.output_dir)

RuntimeError: stack expects each tensor to be equal size, but got [2, 1031330] at entry 0 and [2, 1041733] at entry 1